In [1]:
import pymorphy2
import re
from collections import Counter
morph = pymorphy2.MorphAnalyzer()
dict_type = type(morph.parse('котики')[0].methods_stack[0][0])
import sqlite3

In [2]:
conn = sqlite3.connect('words.db')
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS main 
(word text PRIMARY KEY, 
freq int, 
type text)
""")

conn.commit()

In [3]:
def which_group(word):
    a = morph.parse(word)

    if type(a[0].methods_stack[0][0]) != dict_type:
        return 'несловарно'
    
    elif len(a) == 1:
        return 'однозначно'
    
    else:
        lemma = -1
        pos = -1
        
        for p in a:
            if lemma == -1:
                lemma = p.normal_form
            elif lemma != p.normal_form:
                lemma = 0
                break
        if isinstance(lemma, str):
            lemma = 1

        for p in a:
            if pos == -1:
                pos = str(p.tag.POS)
            elif pos != str(p.tag.POS):
                pos = 0
                break
        if isinstance(pos, str):
            pos = 1
        
        if pos+lemma == 0:
            return 'неоднозначно по части речи и лемме'
        elif pos+lemma == 2:
            return 'неоднозначно по параметрам'
        elif pos == 0 and lemma == 1:
            return 'неоднозначно по части речи'
        else:
            return 'неоднозначно по лемме'

In [4]:
def get_good_text(text):
    return re.findall('\w+-\w+-\w+|\w+-\w+|\w+', text.lower())

In [5]:
def get_cnt(good_text):
    cnt = {}
    top = Counter(good_text).most_common()
    for thing in top:
        cnt[thing[0]] = thing[1]
    return cnt

In [6]:
def fill_db(cnt):
    for word in cnt:
        try:
            cur.execute(
                '''INSERT INTO main (word, freq, type)
                    VALUES (?, ?, ?)''',
                    (word, cnt[word], which_group(word))
                )

        except sqlite3.IntegrityError:

            cur.execute(
            '''select freq
                from main
                where word = (?) ''', (word,)
            )
            old_freq = cur.fetchone()[0]

            cur.execute(
                '''UPDATE main
                    SET freq = (?)
                    WHERE word = (?)''', 
                (cnt[word] + old_freq, word)
                )

    conn.commit()

In [7]:
def dump_file(file):
    with open(file, 'r', encoding='cp1251') as f:
        text = f.read()
    fill_db(get_cnt(get_good_text(text)))

In [12]:
for i in range(10):
    st = 'sp0' + str(i)
    print(st)
    dump_file(st)

sp00
sp01
sp02
sp03
sp04
sp05
sp06
sp07
sp08
sp09


In [13]:
for i in range(10, 90):
    st = 'sp' + str(i)
    print(st)
    dump_file(st)

sp10
sp11
sp12
sp13
sp14
sp15
sp16
sp17
sp18
sp19
sp20
sp21
sp22
sp23
sp24
sp25
sp26
sp27
sp28
sp29
sp30
sp31
sp32
sp33
sp34
sp35
sp36
sp37
sp38
sp39
sp40
sp41
sp42
sp43
sp44
sp45
sp46
sp47
sp48
sp49
sp50
sp51
sp52
sp53
sp54
sp55
sp56
sp57
sp58
sp59
sp60
sp61
sp62
sp63
sp64
sp65
sp66
sp67
sp68
sp69
sp70
sp71
sp72
sp73
sp74
sp75
sp76
sp77
sp78
sp79
sp80
sp81
sp82
sp83
sp84
sp85
sp86
sp87
sp88
sp89


в sp9001, sp9002, sp9017 что-то сломалось

In [19]:
for i in range(3, 10):
    st = 'sp900' + str(i)
    print(st)
    dump_file(st)

sp9003
sp9004
sp9005
sp9006
sp9007
sp9008
sp9009


In [22]:
for i in range(10, 17):
    st = 'sp90' + str(i)
    print(st)
    dump_file(st)

sp9010
sp9011
sp9012
sp9013
sp9014
sp9015
sp9016
